# CMS SynPuf: How to extract medical conditions based on ICD-10 codes

This notebook queries the [CMS SynPuf dataset](https://console.cloud.google.com/marketplace/product/hhs/synpuf?pli=1), a public synthetic patient data in OMOP. This notebook is intended to be used as an example for how to query the public OMOP dataset, and how to do basic visualizations.

> If you are **previewing** this notebook from Verily Workbench, please create a cloud environment and look for this file in the `~/repos/terra-axon-examples/omop_examples/` directory. Instructions for creating a cloud environment are available in the workspace description.

## Import python libraries

In [ ]:
import pandas as pd
from google.cloud import bigquery

# Enable IPython to display matplotlib graphs.
import matplotlib.pyplot as plt
%matplotlib inline

## Notebook setup

In [ ]:
'''
Resolves BQ dataset from reference in workspace.
'''
def get_bq_dataset_from_reference(resource_name):
    BQ_CMD_OUTPUT = !terra resolve --name={resource_name}
    BQ_DATASET = BQ_CMD_OUTPUT[0]
    return BQ_DATASET

## Connect to the BQ database

In [ ]:
# The following line resolves the workspace resource named cms_synthetic_patient_data_omop. 
BQ_dataset = get_bq_dataset_from_reference('cms_synthetic_patient_data_omop')
# The above line will fail if you don't have this resource in your workspace.

# If that is the case, you can hard code the BQ_dataset instead by uncommenting the following line. 
# BQ_dataset = 'bigquery-public-data.cms_synthetic_patient_data_omop'

In [ ]:
job_query_config = bigquery.QueryJobConfig(default_dataset=BQ_dataset)
client = bigquery.Client(default_query_job_config=job_query_config)

## Example for history of skin conditions

The purpose of this example is to calculate the number of participants with a diagnosis of skin condition based on a list of ICD-10 codes.
As a starting point for this study, we define the diagnosis of skin condition as a condition coded with an ICD-10 code from the following list:
- `L70.*`: Acne
- `L20.*`: Atopic dermatitis
- `L40.*`: Psoriasis
- `L80.*`: Vitiligo

where "\*" corresponds to any sequence of numbers.

For this purpose, it is necessary to perform the following steps:
1. **Converting the ICD-10 codes of skin condition to concept IDs**:
This step consists of finding the concept IDs associated with the ICD-10 codes in the following list: `L70`, `L20`, `L40` and `L80`. To do this, we will use the `concept` table of the OMOP vocabulary and find all `concept_code` equal to one of the ICD-10 codes for the `vocabulary_id` equal to "ICD10CM". See the CTE `skin_condition_ICD_concept_ids` in the SQL query below.

2. **Find the standard concept IDs linked to the ICD-10 concept IDs**:
The majority of conditions are coded with a standard concept ID (corresponding to a SNOMED code). Therefore, we need to find the standard concept IDs corresponding to the concept IDs previously extracted in step 1. The CTE `standard_concept_ids` in the SQL query below consists of retrieving only the standard concept IDs from the `concept` table (i.e., `standard_concept` equal to "S") and the CTE `skin_condition_standard_concept_ids` consists of mapping the concept IDs from step 1 to the standard concept IDs (i.e., using the `concept_relationship` table with `relationship_id` equal to "Maps to").

3. **Find all concept IDs linked to the standard skin condition concept IDs**:
The dataset may also contain non-standard concept IDs. To ensure that we include patients who have a diagnosis of skin condition using a non-standard concept ID, we need to find all concept IDs associated with the core set of standard concept IDs identified in the previous step. To do this, we again use the `concept_relationship` table, setting our standard concept IDs as concept_id_2 when using the `relationship_id` of "Maps to". We union these with our standard concept IDs to get a full set of skin condition concept_ids. See the CTEs `skin_condition_nonstandard_concept_ids` and `skin_condition_all_concept_ids` in the SQL query below.

4. **Find all the descendants of the skin condition concept IDs**:
The condition concept IDs are organized under an ontology with different levels of precision. Therefore, to capture all concept IDs for skin condition, we must also find all descendants of the concept IDs previously extracted in step 3. To do this, we will use the `concept_ancestor` table with the `ancestor_concept_id` equal to the concept IDs from step 3 and extract all `descendant_concept_id` as the final list `all_concept_ids_with_descendants` in the SQL query below.

5. **Calculate the number of participants with a diagnosis of skin condition**:
Finally, the last step is to extract and count the participants with at least one diagnosis of skin condition. We will use the `condition_occurrence` table and filter only the conditions coded with a `condition_concept_id` corresponding to a concept ID of the previously extracted list. See the CTE `nb_of_participants_diagnosed_with_skin_condition` in the SQL query below.

6. **Calculate the percentage of participants with a diagnosis of skin condition**:
Finally, the last step is to calculate the percentage with at least one diagnosis of skin condition out of the total number of participants. We will use the number of participants in the `person` table and calculate the percentage. See the CTE `nb_total_of_participants` in the SQL query below.

In [ ]:
query = """
    WITH skin_condition_ICD_concept_ids AS (
        SELECT 
            concept_id,
            CASE concept_code
                WHEN 'L70' THEN 'Acne'
                WHEN 'L20' THEN 'Atopic dermatitis'
                WHEN 'L40' THEN 'Psoriasis'
                ELSE 'Vitiligo'
            END AS skin_condition
        FROM
            `concept`
        WHERE
            concept_code IN ('L70', 'L20', 'L40', 'L80')
            AND vocabulary_id = 'ICD10CM'
    ),
    standard_concept_ids AS (
        SELECT 
            concept_id
        FROM
            `concept`
        WHERE
            standard_concept = 'S'
    ),
    skin_condition_standard_concept_ids AS (
        SELECT
            skin_condition,
            standard_concept_ids.concept_id
        FROM
            skin_condition_ICD_concept_ids
        INNER JOIN
            `concept_relationship`
        ON
            skin_condition_ICD_concept_ids.concept_id = concept_id_1
        INNER JOIN
            standard_concept_ids
        ON
            standard_concept_ids.concept_id = concept_id_2
        WHERE
            relationship_id = 'Maps to'
    ),
    skin_condition_nonstandard_concept_ids AS (
        SELECT
            skin_condition,
            concept_id_1 AS concept_id
        FROM
            `concept_relationship`
        INNE JOIN
            skin_condition_standard_concept_ids
        ON
            relationship_id = 'Maps to'
            AND concept_id_2 = concept_id
    ),
    skin_condition_all_concept_ids AS (
        SELECT DISTINCT
            skin_condition,
            concept_id
        FROM (
            SELECT
                *
            FROM
                skin_condition_standard_concept_ids
        ) UNION ALL (
            SELECT
                *
            FROM
                skin_condition_nonstandard_concept_ids
        )
    ),
    skin_condition_all_concept_ids_with_descendants AS (
        SELECT
            skin_condition,
            descendant_concept_id AS concept_id
        FROM
            skin_condition_all_concept_ids
        INNER JOIN
            `concept_ancestor`
        ON
            concept_id = ancestor_concept_id
    ),
    nb_of_participants_diagnosed_with_skin_condition AS (
        SELECT
            skin_condition,
            COUNT(DISTINCT person_id) AS nb_of_participants_with_skin_condition
        FROM
            `condition_occurrence`
        INNER JOIN
            skin_condition_all_concept_ids_with_descendants
        ON
            condition_concept_id = concept_id
        GROUP BY 
            skin_condition
    ),
    nb_total_of_participants AS (
        SELECT
            COUNT(DISTINCT person_id) AS nb_of_participants
        FROM
            `person`
    )
    SELECT
        skin_condition,
        100*nb_of_participants_with_skin_condition/nb_of_participants AS percentage_of_participants,
    FROM
        nb_total_of_participants,
        nb_of_participants_diagnosed_with_skin_condition
"""

# Execute query
The below code will send a request to BigQuery to execute the query. The results will be stored in a Pandas dataframe.

In [ ]:
df = client.query(query).result().to_dataframe()
df

# Plot visualization
The below code uses matplotlib to plot a simple histogram of the results.

In [ ]:
ax = df.plot.bar(x='skin_condition', y='percentage_of_participants', title='Percentage of participants with a diagnosis of skin condition', rot=0, ylabel='', xlabel='', legend=False)
# Add bar labels
for p in ax.patches:
    ax.annotate('%.2f%%' % (p.get_height()), (p.get_x()+0.07, p.get_height()+0.03))
plt.show()